## Sandbox for interacting with the ARASAAC API

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [ ]:
base_url = "https://api.arasaac.org/api"

def get_all_pictograms(language="en"):
    url = f"{base_url}/pictograms/{language}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [ ]:
def get_pictos(pictogram_data, out_dir="data/picto_files")



In [ ]:
def get_metadata(pictogram_data):

In [ ]:
def print_grid(pico_files, out_file="layout.png", grid=(5, 5)):
  